# Module 13: Ochestrator Agent


이제 마지막으로 강력한 전문 도구 팀을 관리하는 마스터 오케스트레이터를 구축할 것이다. 사용자의 요청에 따라 복잡하고 자율적인 계획 워크플로우를 트리거하거나 게스트 목록 관리 또는 모의 이메일 전송과 같은 작업을 위한 간단한 대화형 도구로 전환할 수 있다.
사용자는 마스터 오케스트레이터하고만 상호작용하며, 마스터 오케스트레이터는 지능적으로 작업을 적절한 하위 시스템에 위임한다.

## 라이브러리 불러오기

먼저, 이 노트북이 빌드된 특정 버전의 Google Agent Development Kit (ADK)를 설치한다. 버전을 고정하면 코드가 항상 예상대로 작동한다.

In [11]:
import json
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown

from google.adk.agents import Agent,SequentialAgent, LoopAgent, ParallelAgent
from google.adk.tools import google_search, ToolContext
from google.adk.tools.agent_tool import AgentTool
from google.adk.sessions import Session, InMemorySessionService
from google.genai.types import Content, Part
from google.adk.runners import Runner

## API 키 구성

다음으로 Google API 키를 안전하게 제공해야 한다. 이 코드는 키를 붙여넣을 수 있는 보안 입력 프롬프트를 생성한다. 그런 다음 키를 환경 변수로 설정하며, 이는 ADK가 요청을 인증하는 표준 방식이다.

In [12]:
load_dotenv()
MODEL = "gemini-2.5-flash"

## 3. 모든 사용자 정의 도구 정의

여기서는 다양한 전문가 에이전트가 전체 워크플로에서 필요로 하는 모든 사용자 정의 파이썬 함수를 정의한다.

In [13]:
# --- 데이터 저장소 ---
GUEST_DATABASE = {}

# --- 도구 함수 ---
def add_guest(name: str, email: str) -> str:
  """게스트의 이름과 이메일을 이벤트 게스트 목록에 추가한다."""
  print(f"  [도구 호출] 게스트 '{name}' (이메일: '{email}') 추가 중.")
  GUEST_DATABASE[email] = name
  return f"{name}을(를) 게스트 목록에 성공적으로 추가했다."

def get_guest_list(tool_context: ToolContext) -> str:
  """이벤트에 등록된 모든 게스트의 현재 목록을 가져온다."""
  print("  [도구 호출] 게스트 목록 검색 중.")
  if not GUEST_DATABASE:
    return "게스트 목록이 현재 비어 있다."
  tool_context.state["guest_list"] = GUEST_DATABASE
  return json.dumps(GUEST_DATABASE)

def sum_costs(costs: list[float]) -> float:
  """숫자 목록의 합계를 계산한다."""
  return sum(costs)

COMPLETION_PHRASE = "계획이 예산 범위 내에 있다."

def exit_loop(tool_context: ToolContext):
  """계획이 승인되고 예산 범위 내에 있을 때만 이 함수를 호출한다."""
  print("  [도구 호출] 예산 승인됨. 루프 종료.")
  # 에스컬레이션 플래그를 설정하여 루프를 탈출한다
  tool_context.actions.escalate = True
  return "예산 승인됨. 계획 확정 중."

def update_session_state(
    tool_context: ToolContext, event_type: str, city: str, budget: int
) -> str:
  """추출된 이벤트 매개변수를 세션 상태에 저장한다."""
  tool_context.state['event_type'] = event_type
  tool_context.state['city'] = city
  tool_context.state['budget'] = budget
  return "세션 상태가 이벤트 매개변수로 업데이트되었다."

def send_mock_email(tool_context: ToolContext) -> str:
    """
    이메일 전송을 시뮬레이션한다. 세션 상태에서 구조화된 EmailDraft 객체를 가져와
    모의 이메일 형식을 지정한다.
    """
    print("  [도구 호출] 모의 이메일 전송 준비 중...")

    guest_list = GUEST_DATABASE
    # 상태에서 구조화된 EmailDraft 객체를 직접 가져온다
    email_draft = tool_context.state.get("email_draft")

    if not guest_list:
        return "이메일을 보낼 게스트가 목록에 없다."

    print("\n--- 이메일 전송 시뮬레이션 ---")
    for email, name in guest_list.items():
        print(f"받는 사람: {name} <{email}>")
        print(f"제목: {email_draft['subject']}")
        print("\n본문:\n" + email_draft["body"])
        print("-----------------------------")

    return f"{len(guest_list)}명의 게스트를 위한 모의 이메일이 성공적으로 준비되었다."

## 4. 호환성을 위한 도구 래퍼 생성

에이전트가 내장 도구와 사용자 정의 함수를 모두 안정적으로 사용할 수 있도록 Google 검색을 "도구 에이전트"로 래핑한다.

In [14]:
# Google 검색 도구를 감싸는 래퍼 에이전트
google_search_agent = Agent(
    name="Google_Search_Agent",
    model=MODEL,
    instruction="당신은 구글 검색 도구를 위한 래퍼일 뿐이다.",
    tools=[google_search]
)

# 에이전트를 도구로 변환
google_search_tool = AgentTool(agent=google_search_agent)

## 5. 모든 전문가 에이전트 정의

이제 전체 전문가 에이전트 팀을 정의하고, 이들이 모두 일관된 `event_type` 변수를 사용하도록 한다.

In [15]:
# --- 섭취(Intake) 에이전트 ---
intake_agent = Agent(
    name="intake_agent",
    model=MODEL,
    instruction="사용자 쿼리에서 이벤트 유형(예: 'AI 기술 모임'), 도시, 예산을 식별하라. 그런 다음 반드시 `update_session_state` 도구를 호출해야 한다.",
    tools=[update_session_state]
)

# --- 게스트 관리 에이전트 ---
guest_management_agent = Agent(
    name="guest_management_agent",
    model=MODEL,
    instruction="당신은 게스트 관리 어시스턴트다. `add_guest`와 `get_guest_list` 도구를 사용하라.",
    tools=[add_guest, get_guest_list]
)

# --- 병렬 물류 스카우트 팀 ---
venue_scout_agent = Agent(
    name="venue_scout_agent",
    model=MODEL,
    instruction="""
    당신은 장소 스카우트다. {{city}}에서 열리는 {{event_type}}을 위한 3곳의 잠재적 장소를 찾아라. 각 장소에 대해 예상 대관료를 찾아라.
    오직 발견한 내용이 담긴 JSON 객체만 출력하라. 예: {"venues": [{"name": "Venue A", "estimated_cost": 5000}]}
    """,
    tools=[google_search],
    output_key="venue_options"
)
catering_scout_agent = Agent(
    name="catering_scout_agent",
    model=MODEL,
    instruction="""
    당신은 케이터링 스카우트다. {{city}}에서 열리는 {{event_type}}에 적합한 3곳의 케이터링 업체를 찾아라. 각 업체에 대해 예상 비용을 찾아라.
    오직 발견한 내용이 담긴 JSON 객체만 출력하라. 예: {"caterers": [{"name": "Caterer A", "estimated_cost": 4000}]}
    """,
    tools=[google_search],
    output_key="catering_options"
)
entertainment_scout_agent = Agent(
    name="entertainment_scout_agent",
    model=MODEL,
    instruction="""
    당신은 엔터테인먼트 스카우트다. {{city}}에서 열리는 {{event_type}}을 위한 3가지 엔터테인먼트 옵션을 찾아라. 각 옵션에 대해 예상 섭외비를 찾아라.
    오직 발견한 내용이 담긴 JSON 객체만 출력하라. 예: {"entertainment": [{"name": "DJ A", "estimated_cost": 1500}]}
    """,
    tools=[google_search],
    output_key="entertainment_options"
)
initial_plan_synthesizer_agent = Agent(
    name="initial_plan_synthesizer_agent",
    model=MODEL,
    instruction="""
    당신은 초기 계획 통합자다. {{venue_options}}, {{catering_options}}, {{entertainment_options}}의 결과를 하나의 JSON 객체로 결합하라.
    최종 JSON에는 "venues", "caterers", "entertainment" 키가 있어야 한다. 오직 JSON 객체만 출력하라.
    예: {"venues": [...], "caterers": [...], "entertainment": [...]}
    """,
    output_key="current_plan"
)

# --- 예산 최적화 팀 (지침 업데이트됨) ---
accountant_agent = Agent(
    name="accountant_agent",
    model=MODEL,
    tools=[sum_costs],
    instruction=f"""
    당신은 꼼꼼한 회계사다. 예산은 {{budget}}이다. 현재 옵션 목록은 {{current_plan}}에 있다.

    당신의 임무는 가장 비용 효율적인 계획을 세우고 평가하는 것이다:
    1. 'venues', 'caterers', 'entertainment' 목록에서 각 카테고리별로 가장 저렴한 옵션을 단 하나씩만 선택하라.
    2. 이 세 가지 항목만으로 구성된 "cheapest_plan"을 생성하라.
    3. 이 "cheapest_plan"에 대해 `sum_costs` 도구를 사용하라.

    - 만약 총비용 > {{budget}}이면, 출력은 반드시 'critique'와 당신이 만든 'cheapest_plan'이 포함된 JSON 객체여야 한다.
    - 그 외의 경우(예산 내), 출력은 반드시 'critique'가 '{COMPLETION_PHRASE}'이고 'cheapest_plan'을 포함하는 JSON 객체여야 한다.

    예산 초과 시 출력 예시:
    ```json
    {{
        "critique": "가장 저렴한 옵션으로도 예산을 초과했습니다. 더 저렴한 장소를 찾으세요.",
        "cheapest_plan": {{
            "venue": {{"name": "Affordable Hall", "estimated_cost": 3200}},
            "caterer": {{"name": "Good Eats Catering", "estimated_cost": 4000}},
            "entertainment": {{"name": "Local DJ", "estimated_cost": 1000}}
        }}
    }}
    ```
    """,
    output_key="evaluation"
)
cost_cutter_agent = Agent(
    name="cost_cutter_agent",
    model=MODEL,
    tools=[google_search_tool, exit_loop],
    instruction=f"""
    당신은 비용 절감 전문가다. {{evaluation}} 변수에 평가 내용을 받았다.

    당신의 임무:
    1. {{evaluation}}의 'critique' 필드를 확인하라.
    2. 만약 비평이 '{COMPLETION_PHRASE}'라면, 반드시 `exit_loop` 도구를 호출해야 한다.
    3. 그렇지 않다면, 계획이 예산을 초과한 것이다. 수정할 계획은 {{evaluation}}의 'cheapest_plan' 필드에 있다.
    4. 'critique'를 읽고 교체할 항목을 식별하라.
    5. 검색 도구를 사용하여 해당 항목에 대한 더 저렴한 대안을 하나 찾아라.
    6. 이 하나의 변경 사항을 포함하고 다른 항목은 그대로 유지한 새로운 전체 JSON 계획을 출력하라.
    """,
    output_key="current_plan"
)

from pydantic import BaseModel, Field

# 통신 에이전트 출력의 구조 정의
class EmailDraft(BaseModel):
  """이메일 초안의 구성 요소를 담는 모델이다."""
  subject: str = Field(description="이메일의 매력적인 제목이다.")
  body: str = Field(description="전체 형식을 갖춘 이메일 본문이다.")

# --- 커뮤니케이션 에이전트 ---
communications_agent = Agent(
    name="communications_agent",
    model=MODEL,
    instruction="""
    당신은 커뮤니케이션 전문가다. {{current_plan}}에서 이벤트 계획을 가져와라.

    당신의 임무는 전문적인 공지 이메일 초안을 작성하고 이를 JSON 객체로 포맷하는 것이다.
    JSON 객체는 "subject"와 "body" 두 개의 키를 가져야 한다.

    **중요: 전체 출력은 오직 원시 JSON 객체여야 한다. `{`로 시작해서 `}`로 끝나야 한다. ```json이나 다른 텍스트, 서론, 서식을 포함하지 말라.**

    필수 출력 형식 예시:
    {
      "subject": "📢 독점 AI 기술 모임 공지!",
      "body": "AI 애호가 여러분,\\n\\n연결하고, 배우고, 혁신할 준비를 하세요!..."
    }
    """,
    output_key="email_draft",
    output_schema=EmailDraft
)

# --- 최종 보고 에이전트 ---
final_report_agent = Agent(
    name="final_report_agent",
    model=MODEL,
    instruction="""
    당신은 고객을 위해 최종 이벤트 계획을 작성하는 전문 프로젝트 어시스턴트다.
    어조는 전문적이고 명확하며 자신감 있어야 한다.

    다음 정보에 접근할 수 있다:
    - `{{current_plan}}`: 각 카테고리별로 하나의 공급업체가 포함된 최종 예산 승인 계획이다.
    - `{{venue_options}}`, `{{catering_options}}`, `{{entertainment_options}}`: 스카우트된 모든 공급업체의 원본 목록이다.
    - `{{budget}}`: 고객의 초기 예산이다.

    당신의 임무는 다음의 정확한 구조를 따르는 포괄적인 마크다운 보고서를 생성하는 것이다:

    1.  **경영 요약:** 이벤트 세부 정보(예: 이벤트 유형, 도시)를 확인하는 간단한 단락으로 시작하고, 예산 ${{budget}} 내에서 비용 효율적인 계획을 찾았음을 명시하라.
    2.  **승인된 계획 및 비용:** `{{current_plan}}`에서 승인된 장소, 케이터링, 엔터테인먼트 공급업체를 개별 비용과 함께 명확하게 나열하는 섹션을 만들어라. 총 최종 비용을 계산하여 표시하라.
    3.  **대안 공급업체 옵션:** `{{venue_options}}`, `{{catering_options}}`, `{{entertainment_options}}`에서 고려되었던 *다른* 공급업체를 나열하는 섹션을 만들어라. 이는 고객에게 참고할 수 있는 추가 옵션을 제공한다.
    4.  **다음 단계:** 다음 단계에 대한 간략하고 전문적인 문구로 보고서를 마무리하라(예: "다음 단계는 예약을 확정하기 위해 이 공급업체들에게 연락하는 것입니다.").

    전체 출력은 마크다운 보고서여야 한다. 다른 대화형 텍스트를 추가하지 말라.
    """,
)

## 6. 모든 워크플로 에이전트 조립

이제 전문가들을 강력한 다단계 워크플로 에이전트로 구성한다.

In [16]:
# --- 병렬 물류 스카우트 워크플로 ---
parallel_logistics_scout = ParallelAgent(
    name="parallel_logistics_scout",
    sub_agents=[venue_scout_agent, catering_scout_agent, entertainment_scout_agent]
)

initial_planning_workflow = SequentialAgent(
    name="initial_planning_workflow",
    sub_agents=[parallel_logistics_scout, initial_plan_synthesizer_agent]
)


# --- 예산 최적화 워크플로 (제안자 없음) ---
budget_refinement_loop = LoopAgent(
    name="budget_refinement_loop",
    sub_agents=[accountant_agent, cost_cutter_agent],
    max_iterations=3
)
budget_optimizer_workflow = SequentialAgent(
    name="budget_optimizer_workflow",
    sub_agents=[budget_refinement_loop]
)

# --- 전체 자율 계획 워크플로 ---
full_plan_workflow = SequentialAgent(
    name="full_plan_workflow",
    sub_agents=[
        intake_agent,
        initial_planning_workflow,
        budget_optimizer_workflow,
        final_report_agent
    ]
)

## 7. 마스터 오케스트레이터 구축

이것은 최종적인 사용자 대면 "지니"다. 유일한 임무는 강력한 도구 팀에 작업을 위임하는 것이다.

In [17]:
# --- 필요한 모든 에이전트/워크플로를 도구로 래핑 ---
full_plan_workflow_tool = AgentTool(agent=full_plan_workflow)
guest_list_manager_tool = AgentTool(agent=guest_management_agent)
communications_tool = AgentTool(agent=communications_agent)

# --- 마스터 오케스트레이터 ---
master_orchestrator_agent = Agent(
    name="master_orchestrator_agent",
    model=MODEL,
    instruction="""
    당신은 마스터 이벤트 기획 어시스턴트다. 올바른 도구에 작업을 위임하라.
    - 상위 수준의 계획 요청(예산 책정, 공급업체 찾기)에는 `full_plan_workflow_tool`을 사용하라.
    - 간단한 게스트 관리(추가, 보기)에는 `guest_list_manager_tool`을 사용하라.
    - 공지 이메일 초안을 작성하려면 `communications_tool`을 사용하라.
    - 최종 이메일을 보내려면 `send_mock_email`을 사용하라.

    - **중요**: `communications_tool`이 사용되면 출력은 'subject'와 'body'가 있는 JSON 객체일 것이다. 당신은 이것을 사용자를 위해 깔끔하고 읽기 쉬운 이메일 초안으로 포맷해야 한다. 원시 JSON을 그대로 보여주지 말라.
    """,
    tools=[
        full_plan_workflow_tool,
        guest_list_manager_tool,
        communications_tool,
        send_mock_email
    ]
)

## 8. 실행 엔진 구축

이것은 쿼리를 실행하기 위한 표준 헬퍼 함수이며, Runner를 처리하고 최종 응답을 스트리밍한다.

In [18]:
async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str):
    """Runner를 초기화하고 주어진 에이전트 및 세션에 대해 쿼리를 실행한다."""
    print(f"\n에이전트 쿼리 실행 중: '{agent.name}', 세션: '{session.id}'...")
    runner = Runner(agent=agent, session_service=session_service, app_name=agent.name)
    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"오류 발생: {e}"

    print("\n" + "-"*50)
    print("최종 응답:")
    display(Markdown(final_response))
    print("-"*50 + "\n")
    return final_response

## 9. 세션 초기화 및 워크플로 실행

마지막으로 `InMemorySessionService`를 설정한다.

In [19]:
# --- 세션 서비스 초기화 ---
# 이 하나의 서비스가 이 노트북의 모든 다른 세션을 관리한다.
session_service = InMemorySessionService()
user_id = "adk_event_planner_001"

## 10. 그랜드 피날레: 전체 대화 실행

마지막으로 세션 서비스를 초기화하고 "자율 이벤트 지니"와 전체 멀티턴 대화를 실행한다.

In [20]:
async def run_capstone_project():
    session = await session_service.create_session(
        app_name=master_orchestrator_agent.name,
        user_id=user_id
    )

    # --- 대화 시작 ---
    # 한국어 환경에 맞게 서울, 500만원 예산으로 수정함
    query1 = "서울에서 예산 500만원으로 50명 규모의 AI 기술 모임을 기획해야 한다. 공급업체를 찾아줘."
    print(f"사용자: {query1}")
    await run_agent_query(master_orchestrator_agent, query1, session, user_id)

    query2 = "멋지다. 게스트 목록에 'Grace Hopper'를 추가해줘. 이메일은 grace@example.com이다."
    print(f"사용자: {query2}")
    await run_agent_query(master_orchestrator_agent, query2, session, user_id)

    query3 = "이제 최종 계획을 바탕으로 공지 이메일 초안을 작성해줘."
    print(f"사용자: {query3}")
    await run_agent_query(master_orchestrator_agent, query3, session, user_id)

    query4 = "완벽하다. 게스트 목록에 이메일을 보내줘."
    print(f"사용자: {query4}")
    await run_agent_query(master_orchestrator_agent, query4, session, user_id)

# 전체 시스템 실행
await run_capstone_project()

사용자: 서울에서 예산 500만원으로 50명 규모의 AI 기술 모임을 기획해야 한다. 공급업체를 찾아줘.

에이전트 쿼리 실행 중: 'master_orchestrator_agent', 세션: 'df1bc335-dbf1-4884-9739-49d0d0721980'...

--------------------------------------------------
최종 응답:


오류 발생: Missing key inputs argument! To use the Google AI API, provide (`api_key`) arguments. To use the Google Cloud API, provide (`vertexai`, `project` & `location`) arguments.

--------------------------------------------------

사용자: 멋지다. 게스트 목록에 'Grace Hopper'를 추가해줘. 이메일은 grace@example.com이다.

에이전트 쿼리 실행 중: 'master_orchestrator_agent', 세션: 'df1bc335-dbf1-4884-9739-49d0d0721980'...

--------------------------------------------------
최종 응답:


오류 발생: Missing key inputs argument! To use the Google AI API, provide (`api_key`) arguments. To use the Google Cloud API, provide (`vertexai`, `project` & `location`) arguments.

--------------------------------------------------

사용자: 이제 최종 계획을 바탕으로 공지 이메일 초안을 작성해줘.

에이전트 쿼리 실행 중: 'master_orchestrator_agent', 세션: 'df1bc335-dbf1-4884-9739-49d0d0721980'...

--------------------------------------------------
최종 응답:


오류 발생: Missing key inputs argument! To use the Google AI API, provide (`api_key`) arguments. To use the Google Cloud API, provide (`vertexai`, `project` & `location`) arguments.

--------------------------------------------------

사용자: 완벽하다. 게스트 목록에 이메일을 보내줘.

에이전트 쿼리 실행 중: 'master_orchestrator_agent', 세션: 'df1bc335-dbf1-4884-9739-49d0d0721980'...

--------------------------------------------------
최종 응답:


오류 발생: Missing key inputs argument! To use the Google AI API, provide (`api_key`) arguments. To use the Google Cloud API, provide (`vertexai`, `project` & `location`) arguments.

--------------------------------------------------

